## Suppress annoying warnings

In [1]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", RuntimeWarning)

## Import stuff that we gonna need

In [2]:
import os
import gzip
import contextlib
import numpy as np
import pandas as pd
from collections import Counter
from pybloom_live import ScalableBloomFilter

from nlabpy.parse.seq import parse_fastq
from nlabpy.utils.mprun import parallel_run

## Build a dataframe with primers from `.csv` file in `../ref`

In [3]:
ls -lah ../ref

total 24K
drwxrwsr-x 2 ilya lab 4.0K Jul 30 08:35 ./
drwxrwsr-x 7 ilya lab 4.0K Jul 30 08:37 ../
lrwxrwxrwx 1 ilya lab   21 Jul 30 08:35 hHSR1 -> /mnt/brick1/ref/hHSR1/
-rw-rw-r-- 1 ilya lab 6.3K Jul 30 08:34 hHSR1_primer_set.csv
-rw-rw-r-- 1 ilya lab 6.7K Jul 30 08:34 hHSR_primer_set.optimized.csv


In [4]:
pdf = pd.read_csv('../ref/hHSR_primer_set.optimized.csv')

def _pos(rec):
    *_, pos = rec['Name'].split('p')
    return int(pos)

def _primer(rec):
    *_, p = rec['Sequence'].split('NNNNNNNNNNNN')
    return p

pdf['position'] = pdf.apply(_pos, axis=1)
pdf['primer'] = pdf.apply(_primer, axis=1)
pdf['count'] = 0
pdf

,Well Position,Name,Sequence,position,primer,count
0,A1,hHSRp182,GTAATACGACTCACTATAGGNNNNNNNNNNNNGCGGATTCAACCGG...,182,GCGGATTCAACCGGTCGATGCAACA,0
1,A2,hHSRp193,GTAATACGACTCACTATAGGNNNNNNNNNNNNTGCTTATGGCGGCG...,193,TGCTTATGGCGGCGGATTCAACCGG,0
2,A3,hHSRp192,GTAATACGACTCACTATAGGNNNNNNNNNNNNGCTTATGGCGGCGG...,192,GCTTATGGCGGCGGATTCAACCGGT,0
3,A4,hHSRp184,GTAATACGACTCACTATAGGNNNNNNNNNNNNCGGCGGATTCAACC...,184,CGGCGGATTCAACCGGTCGATGCAA,0
4,A5,hHSRp183,GTAATACGACTCACTATAGGNNNNNNNNNNNNGGCGGATTCAACCG...,183,GGCGGATTCAACCGGTCGATGCAAC,0
5,A6,hHSRp7,GTAATACGACTCACTATAGGNNNNNNNNNNNNGTGTAAACCGGTTC...,7,GTGTAAACCGGTTCGGACCTCAATT,0
6,A7,hHSRp181,GTAATACGACTCACTATAGGNNNNNNNNNNNNCGGATTCAACCGGT...,181,CGGATTCAACCGGTCGATGCAACAC,0
7,A8,hHSRp187,GTAATACGACTCACTATAGGNNNNNNNNNNNNTGGCGGCGGATTCA...,187,TGGCGGCGGATTCAACCGGTCGATG,0
8,A9,hHSRp186,GTAATACGACTCACTATAGGNNNNNNNNNNNNGGCGGCGGATTCAA...,186,GGCGGCGGATTCAACCGGTCGATGC,0
9,A10,hHSRp194,GTAATACGACTCACTATAGGNNNNNNNNNNNNCTGCTTATGGCGGC...,194,CTGCTTATGGCGGCGGATTCAACCG,0


## The actual `.fastq` processing function

If used with multiprocessing, `filter_reads` should be given a keyword parameter `queue` where the final stats for the given input data will be put. Your function can take any number or combination of positional and keyword arguments but if you want it to return a value in the multiprocessing settings, pass a `mp.queue` instance as a keyword argument and be sure to `queue.put()` your value before returning from the function.

In [5]:
def hamming(s1, s2):
    '''
    Computes Hamming distance between s1 and s2.
    '''
    if len(s1) != len(s2):
        raise ValueError('{s1} and {s2} must be the same length to compute Hamming distance!'.format(s1=s1, s2=s2))
    return sum(ch1 != ch2 for ch1,ch2 in zip(s1, s2))


def filter_reads(fastq1, fastq2, pdf, mismatch=1, phase=3,
                   anchor='GTAATACGACTCACTATAGG', lprimer=25, lrandom=12, queue=None):
    '''
    Allow anchor mismatch
    '''
    
    def _hamming(rec, seq):
        return hamming(rec['primer'], seq)
    
    
    def write_fastq(key, fq1, fq2):
        def format_fastq(fq):
            fastq_tpl = '@{id}\n{seq}\n+\n{qual}\n'
            return fastq_tpl.format(id=fq[0], seq=fq[1], qual=fq[2])
        output_files[key][0].write(format_fastq(fq1))
        output_files[key][1].write(format_fastq(fq2))

    
    
    lanchor = len(anchor)
    min_len = 4 + lanchor + lrandom + lprimer
    
    primers = set(pdf['primer'])
    dst = pd.DataFrame({'primer': pdf['primer'], 'dist': 0})
    stat = Counter()
    output_files = {}
    umis = ScalableBloomFilter()
    
    with contextlib.ExitStack() as stack:
        # T7 (anchor) sequence not found
        output_files['noanchor'] = (
            stack.enter_context(gzip.open(fastq1.replace('R1', 'R1_noanchor'), 'wt')),
            stack.enter_context(gzip.open(fastq2.replace('R2', 'R2_noanchor'), 'wt'))
        )
        # T7 found but primer is unknown
        output_files['unknown'] = (
            stack.enter_context(gzip.open(fastq1.replace('R1', 'R1_unknown'), 'wt')),
            stack.enter_context(gzip.open(fastq2.replace('R2', 'R2_unknown'), 'wt'))
        )
        # All good
        output_files['good'] = (
            stack.enter_context(gzip.open(fastq1.replace('R1', 'R1_good'), 'wt')),
            stack.enter_context(gzip.open(fastq2.replace('R2', 'R2_good'), 'wt'))
        )
        
        # All primers from T7 sequences
        output_files['T7'] = (
            stack.enter_context(gzip.open(fastq1.replace('R1', 'R1_T7'), 'wt')),
            stack.enter_context(gzip.open(fastq2.replace('R2', 'R2_T7'), 'wt'))
        )
    
        for fqrec1,fqrec2 in zip(parse_fastq(fastq1), parse_fastq(fastq2)):
            stat['total'] += 1
            if len(fqrec1[1]) < min_len or len(fqrec2[1]) < min_len:
                stat['too_short'] += 1
                continue
                
            # Get all possible anchor sequences (due to padding)
            # and their hamming distances
            anchors = {fqrec1[1][i:i+lanchor]: (i, hamming(fqrec1[1][i:i+lanchor], anchor)) 
                       for i in range(phase)}
            idx = None
            for _, d in anchors.items():
                if d[1] < mismatch + 1:
                    stat['anchor'] += 1
                    idx = d[0]
                    break
            
            if idx is not None:    
                start = idx + lanchor + lrandom
                umi = fqrec1[1][idx+lanchor:idx+lanchor+lrandom]
                primer = fqrec1[1][start:start+lprimer]
                
                if not umi in umis:
                    stat['T7'] += 1
                    write_fastq('T7',
                                (fqrec1[0],
                                 fqrec1[1][start:],
                                 fqrec1[2][start:]
                                ),
                                (fqrec2[0],
                                 fqrec2[1][:20],
                                 fqrec2[2][:20]
                                 )
                                )
                    umis.add(umi)
                
                if primer in primers:
                    stat['exact'] += 1
                    write_fastq('good', fqrec1, fqrec2)
                else:
                    dst['dist'] = dst.apply(_hamming, axis=1, args=(primer,))
                    if dst.loc[dst['dist'].argmin(), 'dist'] < 2:
                        # Inexact primer matches still go to the "good" file
                        stat['inexact'] += 1
                        write_fastq('good', fqrec1, fqrec2)
                    else:
                        # No primer found
                        stat['unknown'] += 1
                        write_fastq('unknown', fqrec1, fqrec2)    
            else:
                # T7 (anchor) not found
                stat['NA'] += 1
                write_fastq('noanchor', fqrec1, fqrec2)
            if stat['total'] % 1000000 == 0:
                print('\tprocessed {} records ...\n'.format(stat['total']))
                print(stat)
    if not queue is None:
        queue.put((fastq1, stat))
    return fastq1, stat

## Create an iterable returning the names of `.fastq` files to process

In [6]:
R1_tpl = '../data/IS-234_hsr{:02d}/R1.fastq.gz'
R2_tpl = '../data/IS-234_hsr{:02d}/R2.fastq.gz'
stpl = '\t{key}:\t\t{value}'

data = ((R1_tpl.format(i), R2_tpl.format(i)) for i in range(1,17))

## Run `filter_reads` in parallel over `data`

`parallel_run` creates a `mp.Process` instance for each `filter_reads` call on data provided in `data` iterable (which in this case returns tuples of R1 and R2 filenames). Additional positional and keywords arguments for `filter_reads` are provided in `args` and `kwargs` respectively.

In [7]:
result = parallel_run(filter_reads, data, args=(pdf,), kwargs={'mismatch': 2})

started 16 processes ...


In [8]:
for name,stat in result:
    print("\n\t {}\n".format(name))
    [print(stpl.format(key=k, value=v)) for k,v in stat.items()]


	 ../data/IS-234_hsr08/R1.fastq.gz

	total:		12936
	NA:		9710
	too_short:		1863
	anchor:		1363
	T7:		1025
	inexact:		111
	exact:		1086
	unknown:		166

	 ../data/IS-234_hsr07/R1.fastq.gz

	total:		9659
	NA:		4906
	anchor:		2973
	T7:		1169
	unknown:		314
	exact:		2444
	too_short:		1780
	inexact:		215

	 ../data/IS-234_hsr06/R1.fastq.gz

	total:		17858
	NA:		14632
	too_short:		2233
	anchor:		993
	T7:		461
	exact:		794
	unknown:		99
	inexact:		100

	 ../data/IS-234_hsr05/R1.fastq.gz

	total:		15925
	NA:		10138
	too_short:		3174
	anchor:		2613
	T7:		1942
	exact:		2054
	unknown:		380
	inexact:		179

	 ../data/IS-234_hsr04/R1.fastq.gz

	total:		26493
	NA:		18426
	anchor:		4595
	T7:		3383
	exact:		3705
	too_short:		3472
	unknown:		525
	inexact:		365

	 ../data/IS-234_hsr03/R1.fastq.gz

	total:		36955
	NA:		28617
	too_short:		3467
	anchor:		4871
	T7:		2297
	exact:		4055
	unknown:		469
	inexact:		347

	 ../data/IS-234_hsr12/R1.fastq.gz

	total:		74647
	NA:		71233
	too_short:		1888
	anchor:		152